## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
import numpy as np

A = np.random.randint(0, 1000, size=(1_000_000,))
A

array([298, 484, 297, ..., 115, 839, 485])

In [3]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [4]:
%timeit f1(A)

478 ms ± 5.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%lprun -f f1 f1(A)

In [8]:
def f2(A):
    return sum(A) / len(A) + 100

In [9]:
%timeit f2(A)

159 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
def f3(A):
    return A.mean() + 100

In [11]:
%timeit f3(A)

634 µs ± 2.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
import numba

In [13]:
@numba.njit
def f4(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [14]:
%timeit f4(A)

505 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [15]:
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

g(df)

,col1,col2,col3,col4,key
3,925,13,817,666,a
16,360,433,44,361,a
25,515,224,325,617,a
37,188,351,565,584,a
43,219,298,389,659,a
...,...,...,...,...,...
1999948,65,330,640,641,e
1999958,287,544,590,665,e
1999969,408,563,715,53,e
1999982,59,952,364,951,e


In [16]:
%timeit g(df)

493 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%lprun -f g g(df)

In [17]:
(df['key'] != 'f') & (df['key'] != 'g')

0          False
1           True
2           True
3           True
4           True
           ...  
1999995    False
1999996    False
1999997     True
1999998    False
1999999     True
Name: key, Length: 2000000, dtype: bool

In [18]:
def g1(df):
    return df[(df['key'] != 'f') & (df['key'] != 'g')]

g1(df)

,col1,col2,col3,col4,key
1,170,998,907,3,e
2,631,232,295,940,d
3,925,13,817,666,a
4,994,547,636,796,b
6,226,595,890,327,b
...,...,...,...,...,...
1999992,480,773,34,259,d
1999993,81,973,750,263,d
1999994,438,264,863,659,d
1999997,45,654,481,647,e


In [19]:
%timeit g1(df)

218 ms ± 7.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Лабораторная работа 3

In [1]:
!pip install line_profiler

In [6]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [21]:
recipes = pd.read_csv("./data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')

reviews = pd.read_csv("./data/reviews_sample.csv", sep=",",parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

In [24]:
def func_A(frame):
    counter = 0
    values = 0
    for _, row in frame.iterrows():
        if row["date"].year == 2010:
            values += row["rating"]
            counter += 1

    return values/counter

func_A(reviews)

4.4544402182900615

In [26]:
%timeit func_A(reviews)

6.08 s ± 45.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
def func_B(frame):
    counter = 0
    values = 0
    selected_year_df = frame[frame['date'].dt.year == 2010]
    for index, row in selected_year_df.iterrows():
        values += row["rating"]
        counter += 1

    return values/counter

func_B(reviews)

4.4544402182900615

In [28]:
%timeit func_B(reviews)

583 ms ± 3.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
def func_C(frame):
    selected_year_df = frame['date'].dt.year == 2010
    return frame.loc[selected_year_df, 'rating'].mean()

func_C(reviews)

4.4544402182900615

In [30]:
%timeit func_C(reviews)

6.74 ms ± 57.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [31]:
%lprun -f func_A func_A(reviews)

Timer unit: 1e-07 s

Total time: 15.5739 s
File: <ipython-input-24-c1ac0507c804>
Function: func_A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def func_A(frame):
     2         1         21.0     21.0      0.0      counter = 0
     3         1         11.0     11.0      0.0      values = 0
     4    126697  142833889.0   1127.4     91.7      for _, row in frame.iterrows():
     5    126696   11913663.0     94.0      7.6          if row["date"].year == 2010:
     6     12094     944382.0     78.1      0.6              values += row["rating"]
     7     12094      46862.0      3.9      0.0              counter += 1
     8                                           
     9         1          8.0      8.0      0.0      return values/counter

In [32]:
%lprun -f func_B func_B(reviews)

Timer unit: 1e-07 s

Total time: 1.47656 s
File: <ipython-input-27-9f3d24b3cf62>
Function: func_B at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def func_B(frame):
     2         1         13.0     13.0      0.0      counter = 0
     3         1          6.0      6.0      0.0      values = 0
     4         1      91531.0  91531.0      0.6      selected_year_df = frame[frame['date'].dt.year == 2010]
     5     12095   13495836.0   1115.8     91.4      for index, row in selected_year_df.iterrows():
     6     12094    1129542.0     93.4      7.6          values += row["rating"]
     7     12094      48636.0      4.0      0.3          counter += 1
     8                                           
     9         1         24.0     24.0      0.0      return values/counter

In [33]:
%lprun -f func_C func_C(reviews)

Timer unit: 1e-07 s

Total time: 0.0072214 s
File: <ipython-input-29-2030648b8643>
Function: func_C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def func_C(frame):
     2         1      62744.0  62744.0     86.9      selected_year_df = frame['date'].dt.year == 2010
     3         1       9470.0   9470.0     13.1      return frame.loc[selected_year_df, 'rating'].mean()

Медленней всего работает первая функция, основное время занимает цикл по iterrows

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [35]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [37]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

In [40]:
%timeit get_word_reviews_count(reviews)

18.6 s ± 120 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<p>1)iterrows сам по себе медленный 
<p>2)Второй цикл совсем не нужен

In [38]:
def get_word_reviews_count_optimized(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        for word in row['review'].split(' '):
            word_reviews[word] = word_reviews.get(word, 0) + 1
    
    return word_reviews

In [39]:
%lprun -f get_word_reviews_count_optimized get_word_reviews_count_optimized(reviews)

In [41]:
%timeit get_word_reviews_count_optimized(reviews)

8.34 s ± 28.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [52]:
buffer = reviews[['recipe_id', 'rating']].dropna()
mask = buffer['rating'] != 0
buffer = buffer[mask].groupby(buffer['recipe_id'])['rating']

In [54]:
def new_func_A(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

In [43]:
@numba.jit(nopython=True)
def new_func_B(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

In [55]:
def new_func_C(A, F):
    results_list = (A - F).abs()
    return 100/len(A) * results_list.sum()

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4